In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.api import SimpleExpSmoothing
import math

Для того, чтобы получить прогнозные значения amount на 90 дней необходимо спрогнозировать погодные условия. Задача: 
1) Подготовить данные, объединив hot, rain, snow and non_hor_rain_snow;
2) Проверить временные ряды на одинаковые значения:  если каждый месяц в каждои городе погода идентична по дням, тогда нет необходиммости строить модель для предсказания погоды.

In [2]:
weather = pd.read_parquet(r"..\all_data\main_data_parq\weather_df.parquet")

In [3]:
weather['year'] = weather.date.dt.strftime('%Y')
weather['month'] = weather.date.dt.strftime('%b')

Создадим период 90 дней, но значения заменим на месяцы

In [4]:

weather['category'] =  weather.rain.replace({1:2}) + weather.snow.replace({1:3}) + weather.hot 
period = pd.date_range(start = weather.date.iloc[-1], periods = 90).strftime('%b')
period

Index(['Sep', 'Sep', 'Sep', 'Sep', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct',
       'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct',
       'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Oct',
       'Oct', 'Oct', 'Oct', 'Oct', 'Oct', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov',
       'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov',
       'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Nov',
       'Nov', 'Nov', 'Nov', 'Nov', 'Nov', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec',
       'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec',
       'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec', 'Dec'],
      dtype='object')

In [5]:
weather

,place,hot,rain,snow,date,year,month,category
0,Фалькония,0,0,0,2216-01-02,2216,Jan,0
1,Анор Лондо,1,0,0,2216-01-02,2216,Jan,1
2,Врата Балдура,0,0,1,2216-01-02,2216,Jan,3
3,Нокрон,0,0,0,2216-01-02,2216,Jan,0
4,Кеджистан,0,0,1,2216-01-02,2216,Jan,3
...,...,...,...,...,...,...,...,...
4995,Фалькония,0,0,0,2218-09-27,2218,Sep,0
4996,Анор Лондо,0,0,1,2218-09-27,2218,Sep,3
4997,Врата Балдура,1,0,0,2218-09-27,2218,Sep,1
4998,Нокрон,0,0,0,2218-09-27,2218,Sep,0


Как мы видим погода не пересекается(в один день нет ни дождя, ни жары, ни снега). Закодируем их как классы
nothing(ни одно из условий)- 0,
hot - 1,
rain - 2,
snow - 3,

In [6]:
for place in weather.place.unique():
    for month in period.unique():
        for yaers in weather.year.unique():
            r = pd.DataFrame(weather[(weather.place == place) & (weather.month == month ) & (weather.year == yaers )]['category'])
            if len(r) > 1:
                r['cat_2'] = weather[(weather.place == place) & (weather.month == month)& (weather.year == yaers)]['category']
                print(f"{yaers}, {month},{place} :", r['category'].equals(r['cat_2']))

2216, Sep,Фалькония : True
2217, Sep,Фалькония : True
2218, Sep,Фалькония : True
2216, Oct,Фалькония : True
2217, Oct,Фалькония : True
2216, Nov,Фалькония : True
2217, Nov,Фалькония : True
2216, Dec,Фалькония : True
2217, Dec,Фалькония : True
2216, Sep,Анор Лондо : True
2217, Sep,Анор Лондо : True
2218, Sep,Анор Лондо : True
2216, Oct,Анор Лондо : True
2217, Oct,Анор Лондо : True
2216, Nov,Анор Лондо : True
2217, Nov,Анор Лондо : True
2216, Dec,Анор Лондо : True
2217, Dec,Анор Лондо : True
2216, Sep,Врата Балдура : True
2217, Sep,Врата Балдура : True
2218, Sep,Врата Балдура : True
2216, Oct,Врата Балдура : True
2217, Oct,Врата Балдура : True
2216, Nov,Врата Балдура : True
2217, Nov,Врата Балдура : True
2216, Dec,Врата Балдура : True
2217, Dec,Врата Балдура : True
2216, Sep,Нокрон : True
2217, Sep,Нокрон : True
2218, Sep,Нокрон : True
2216, Oct,Нокрон : True
2217, Oct,Нокрон : True
2216, Nov,Нокрон : True
2217, Nov,Нокрон : True
2216, Dec,Нокрон : True
2217, Dec,Нокрон : True
2216, Sep,

Как мы выдим, каждый день  идентичен по погодным дню в прошлым году, то есть значения погодных условий повторяются ежегодно.

In [7]:
weather = weather.set_index('date')

In [8]:
start_date_end_per = pd.date_range(start ='2218-09-28' , periods = 90)

st = pd.to_datetime('2217-09-28')
en = pd.to_datetime('2217-12-26')
trainer = []
for places in weather.place.unique():
    model_weather = pd.DataFrame()
    data = weather[weather.place == places]
    model_weather['date'] = start_date_end_per
    model_weather['place'] = places
    model_weather[['hot','rain','snow']] = data[st:en][['hot','rain','snow']].values
    trainer.append(model_weather)

weather_trainer  = pd.concat(trainer)

In [9]:
weather_trainer

,date,place,hot,rain,snow
0,2218-09-28,Фалькония,0,0,0
1,2218-09-29,Фалькония,1,0,0
2,2218-09-30,Фалькония,1,0,0
3,2218-10-01,Фалькония,1,0,0
4,2218-10-02,Фалькония,0,1,0
...,...,...,...,...,...
85,2218-12-22,Кеджистан,0,0,0
86,2218-12-23,Кеджистан,0,1,0
87,2218-12-24,Кеджистан,0,0,0
88,2218-12-25,Кеджистан,0,0,0


In [10]:
weather_trainer.to_csv(r"..\all_data\data_predictions\weather_trainer.csv", index = False)